In [9]:
# Initial imports
import requests
import os
import pandas as pd
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import seaborn as sns
from MCForecastTools import MCSimulation
import numpy as np
import pandas_ta as ta
import plotly.graph_objects as go
import yfinance as yf

%matplotlib inline

In [10]:
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2"
)

In [23]:
# Tickers
ticker = ["BABA", "DIS", "TSLA", "ETHE", "SPY"]

# set timeframe to '1D'

timeframe = "1D"


# Set start and end datetimes of 5 years
start_date = pd.Timestamp("2016-05-24", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-05-24", tz="America/New_York").isoformat()


# Get 5 year's worth of historical data 
all_stocks = alpaca.get_barset(
    ticker,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df

# # Display sample data
all_stocks.head(10)

BABA                                        DIS  \
                             open     high     low   close    volume    open   
time                                                                           
2017-06-05 04:00:00+00:00  124.00  125.560  123.87  125.37   8021032  104.63   
2017-06-06 04:00:00+00:00  125.60  126.200  124.27  124.63   7974948  106.17   
2017-06-07 04:00:00+00:00  125.08  125.910  124.11  125.67   7319114  105.79   
2017-06-08 04:00:00+00:00  142.51  143.700  135.21  142.38  62224198  103.19   
2017-06-09 04:00:00+00:00  144.57  148.290  137.01  139.36  46078639  104.29   
2017-06-12 04:00:00+00:00  139.28  142.200  136.05  139.10  27531987  105.89   
2017-06-13 04:00:00+00:00  141.81  142.244  135.90  136.45  33762441  107.14   
2017-06-14 04:00:00+00:00  137.80  139.250  135.26  136.68  25584146  104.53   
2017-06-15 04:00:00+00:00  135.64  135.670  133.10  135.01  25781627  103.47   
2017-06-16 04:00:00+00:00  136.28  136.500  133.55  134.90  29647562  106.28   

                                                              ...     SPY  \
                              high      low   close   volume  ...    open   
time                                                          ...           
2017-06-05 04:00:00+00:00  106.875  103.720  106.51  5302320  ...  243.98   
2017-06-06 04:00:00+00:00  106.190  104.610  105.51  8287158  ...  243.31   
2017-06-07 04:00:00+00:00  106.250  103.430  103.52  4621500  ...  243.60   
2017-06-08 04:00:00+00:00  104.490  101.800  104.32  8839933  ...  243.77   
2017-06-09 04:00:00+00:00  105.920  104.080  105.62  7628117  ...  244.09   
2017-06-12 04:00:00+00:00  107.080  103.690  107.05  6753315  ...  243.14   
2017-06-13 04:00:00+00:00  107.260  104.000  104.15  3788305  ...  243.99   
2017-06-14 04:00:00+00:00  104.590  103.370  103.74  3483352  ...  244.86   
2017-06-15 04:00:00+00:00  106.070  102.880  105.95  3685768  ...  242.68   
2017-06-16 04:00:00+00:00  106.350  104.765  105.53  5951970  ...  242.77   

                                                                    TSLA  \
                             high       low    close     volume     open   
time                                                                       
2017-06-05 04:00:00+00:00  244.30  243.7600  243.965   27596403  338.500   
2017-06-06 04:00:00+00:00  243.98  243.1200  243.250   36562468  344.950   
2017-06-07 04:00:00+00:00  243.92  242.8300  243.650   38602639  356.300   
2017-06-08 04:00:00+00:00  244.33  243.1700  243.760   48051333  363.750   
2017-06-09 04:00:00+00:00  245.01  241.9500  243.450  103578275  374.420   
2017-06-12 04:00:00+00:00  243.42  242.3800  243.340   57096569  357.990   
2017-06-13 04:00:00+00:00  244.61  243.5799  244.520   43470532  367.670   
2017-06-14 04:00:00+00:00  244.87  243.2900  244.260   55098826  381.085   
2017-06-15 04:00:00+00:00  243.91  242.3600  243.790   51523068  372.500   
2017-06-16 04:00:00+00:00  242.83  241.6337  242.630   48998243  377.780   

                                                                 
                               high     low     close    volume  
time                                                             
2017-06-05 04:00:00+00:00  348.4400  334.21  347.3200   5684288  
2017-06-06 04:00:00+00:00  359.4929  344.66  352.8000   9465968  
2017-06-07 04:00:00+00:00  360.5000  355.14  359.5500   7850542  
2017-06-08 04:00:00+00:00  371.9000  360.22  370.0100   7489685  
2017-06-09 04:00:00+00:00  376.8700  354.80  357.4500  14323526  
2017-06-12 04:00:00+00:00  364.5000  350.62  358.9900   8528936  
2017-06-13 04:00:00+00:00  376.0000  366.61  375.9500   9720070  
2017-06-14 04:00:00+00:00  384.2500  376.31  380.6599  10303555  
2017-06-15 04:00:00+00:00  375.4600  366.49  375.3400   8585072  
2017-06-16 04:00:00+00:00  378.0100  370.10  371.4000   4691654  

[10 rows x 25 columns]

In [19]:
all_stocks_df = all_stocks.dropna()

In [22]:
# Create and empty DataFrame for closing prices
all_closing_prices = pd.DataFrame()

# Fetch the closing prices 
all_closing_prices["BABA"] = all_stocks["BABA"]["close"]
all_closing_prices["SPY"] = all_stocks["SPY"]["close"]
all_closing_prices["DIS"] = all_stocks["DIS"]["close"]
#all_closing_prices["ETHE"] = all_stocks["ETHE"]["close"]

# Drop the time component of the date
all_closing_prices.index = all_closing_prices.index.date

# Compute daily returns
all_returns = all_closing_prices.pct_change().dropna()
all_returns.tail()

,BABA,SPY,DIS,ETHE


In [17]:
# Calculate Cumulative Returns
all_cumulative_returns = (1 + all_returns).cumprod()

# Plot Cummulative Returns
all_cumulative_returns.plot(title = 'Cumulative Returns', figsize = (20,10) )

NameError: name 'all_returns' is not defined

In [18]:
all_returns.plot.density()

NameError: name 'all_returns' is not defined

In [19]:
# Calculate Standard Deviation
all_std = all_returns.std()

# Calculate Annual Standard Deviation
all_annual_std = all_std * np.sqrt(252)

# Display Standard Deviation
all_annual_std

NameError: name 'all_returns' is not defined

In [20]:
# Calculate Correlation
all_correlation = all_returns.corr()

NameError: name 'all_returns' is not defined

In [22]:
# Plot Correlation 
sns.heatmap(all_correlation, vmin = -1, vmax = 1, annot = True)

NameError: name 'all_correlation' is not defined

In [23]:
# Calculate Covariance
baba_spy_covar = all_returns['BABA'].rolling(window = 60).cov(all_returns['SPY'])
dis_spy_covar = all_returns['DIS'].rolling(window = 60).cov(all_returns['SPY'])
tsla_spy_covar = all_returns['TSLA'].rolling(window = 60).cov(all_returns['SPY'])
ethe_spy_covar = all_returns['ETHE'].rolling(window = 60).cov(all_returns['SPY'])

NameError: name 'all_returns' is not defined

In [24]:
# Calculate Variance
baba_spy_var = all_returns['SPY'].rolling(window = 60).var()
dis_spy_var = all_returns['SPY'].rolling(window = 60).var()
tsla_spy_var = all_returns['SPY'].rolling(window = 60).var()
ethe_spy_var = all_returns['SPY'].rolling(window = 60).var()

NameError: name 'all_returns' is not defined

In [26]:
# Calculate Beta
baba_beta = baba_spy_covar / baba_spy_var
dis_beta = dis_spy_covar / dis_spy_var
tsla_beta = tsla_spy_covar / tsla_spy_var
ethe_beta = ethe_spy_covar / ethe_spy_var
combined_beta = baba_beta + dis_beta + tsla_beta +ethe_beta

NameError: name 'baba_spy_covar' is not defined

In [27]:
# Plot Beta
beta_plot = combined_beta.plot(title= "60 Day Beta", figsize=(20, 10))
bab_beta_plot.legend(["BABA", "DIS", "TSLA", "ETHE"])

NameError: name 'combined_beta' is not defined

In [28]:
# Calculate the Sharpe Ratio
all_sharpe_ratios = (all_returns.mean() * 252) / (all_returns.std() * np.sqrt(252))


NameError: name 'all_returns' is not defined

In [29]:
# Plot Sharpe Ratio
all_sharpe_ratios.plot(kind = "bar", title = "Sharpe Ratio")

NameError: name 'all_sharpe_ratios' is not defined